In [ ]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
from scipy.optimize import minimize

#Au material script from complex refractive index data

wvl_data = np.array([250.000000, 260.000000, 270.000000, 280.000000, 290.000000, 300.000000,
                              310.000000, 320.000000, 330.000000, 340.000000, 350.000000, 360.000000,
                              370.000000, 380.000000, 390.000000, 400.000000, 410.000000, 420.000000,
                              430.000000, 440.000000, 450.000000, 460.000000, 470.000000, 480.000000,
                              490.000000, 500.000000, 510.000000, 520.000000, 530.000000, 540.000000,
                              550.000000, 560.000000, 570.000000, 580.000000, 590.000000, 600.000000,
                              610.000000, 620.000000, 630.000000, 640.000000, 650.000000, 660.000000,
                              670.000000, 680.000000, 690.000000, 700.000000, 710.000000, 720.000000,
                              730.000000, 740.000000, 750.000000, 760.000000, 770.000000, 780.000000,
                              790.000000, 800.000000, 810.000000, 820.000000, 830.000000, 840.000000,
                              850.000000, 860.000000, 870.000000, 880.000000, 890.000000, 900.000000,
                              910.000000, 920.000000, 930.000000, 940.000000, 950.000000, 960.000000,
                              970.000000, 980.000000, 990.000000, 1000.000000])

#Refractive index data
n_data = np.array([1.486293, 1.517101, 1.602219, 1.678100, 1.750429, 1.800042, 1.830000, 1.836011,
                   1.813111, 1.781199, 1.751161, 1.726732, 1.706249, 1.687926, 1.670782, 1.658000,
                   1.641467, 1.626593, 1.607286, 1.572004, 1.503407, 1.417996, 1.315496, 1.189871,
                   1.018554, 0.849238, 0.698231, 0.571152, 0.475555, 0.397920, 0.355713, 0.318417,
                   0.295010, 0.278152, 0.261664, 0.245531, 0.229740, 0.214278, 0.199131, 0.184288,
                   0.169737, 0.164060, 0.162125, 0.160792, 0.160087, 0.160906, 0.161863, 0.162930,
                   0.164154, 0.166357, 0.168557, 0.170750, 0.172934, 0.175425, 0.178189, 0.180917,
                   0.183609, 0.186265, 0.189334, 0.193206, 0.197006, 0.200737, 0.204403, 0.208008,
                   0.211773, 0.215750, 0.219658, 0.223500, 0.227279, 0.230997, 0.234659, 0.239028,
                   0.243770, 0.248435, 0.253026, 0.257546])

#Absorption coefficient data (imaginary coefficient of complex refractive index)
k_data = np.array([1.660980, 1.759202, 1.824512, 1.873471, 1.904443, 1.919298, 1.916000, 1.897531,
                   1.870450, 1.852625, 1.847116, 1.855697, 1.883198, 1.916907, 1.939990, 1.956000,
                   1.957488, 1.949527, 1.934151, 1.910625, 1.878436, 1.843614, 1.814717, 1.799925,
                   1.820950, 1.892178, 2.027084, 2.185897, 2.374957, 2.553141, 2.695668, 2.832206,
                   2.899935, 2.930925, 2.961718, 2.992315, 3.022714, 3.052916, 3.082919, 3.112725,
                   3.142334, 3.293285, 3.477580, 3.652616, 3.817434, 3.959739, 4.097111, 4.230027,
                   4.358280, 4.474892, 4.588542, 4.699445, 4.807792, 4.914824, 5.020584, 5.124162,
                   5.225688, 5.325278, 5.419440, 5.505117, 5.589481, 5.672590, 5.754499, 5.835259,
                   5.919898, 6.009588, 6.097960, 6.185070, 6.270970, 6.355709, 6.439333, 6.519685,
                   6.597783, 6.674968, 6.751271, 6.826722])


#Fit the data to Lorentz Function and generate Au material

def lorentzfunc(p: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Returns the complex ε profile given a set of Lorentzian parameters p
    (σ_0, ω_0, γ_0, σ_1, ω_1, γ_1, ...) for a set of frequencies x.
    """
    N = len(p) // 3
    y = np.zeros(len(x))
    for n in range(N):
        A_n, x_n, g_n = p[3 * n : 3 * n + 3]
        y = y + A_n / (np.square(x_n) - np.square(x) - 1j * x * g_n)
    return y


def lorentzerr(p: np.ndarray, x: np.ndarray, y: np.ndarray) -> float:
    """
    Returns the error (or residual or loss) as the L2 norm
    of the difference of ε(p,x) and y over a set of frequencies x.
    """
    yp = lorentzfunc(p, x)
    val = np.sum(np.square(abs(y - yp)))
    return val


def lorentzfit(
    p0: np.ndarray,
    x: np.ndarray,
    y: np.ndarray,
    tol: float = 1e-25,
    maxeval: float = 10000,
) -> Tuple[np.ndarray, float]:
    """
    Returns the optimal Lorentzian polarizability parameters and error
    which minimize the error in ε(p0,x) relative to y for an initial
    set of Lorentzian polarizability parameters p0 over a set of
    frequencies x using the scipy.optimize algorithm for a relative
    tolerance tol and a maximum number of iterations maxeval.
    """
    result = minimize(
        lambda p: lorentzerr(p, x, y),
        p0,
        method="L-BFGS-B",
        options={"ftol": tol, "maxiter": maxeval},
    )
    popt = result.x
    minf = result.fun
    return popt, minf


if __name__ == "__main__":
    
    n = n_data[:] + 1j * k_data[:]
    eps_inf = 1.1

    eps = np.square(n) - eps_inf

    wl = wvl_data[:]
    wl_min = 250  # minimum wavelength (units of nm)
    wl_max = 1000  # maximum wavelength (units of nm)
    start_idx = np.where(wl > wl_min)
    idx_start = start_idx[0][0]
    end_idx = np.where(wl < wl_max)
    idx_end = end_idx[0][-1] + 1

    # The fitting function is ε(f) where f is the frequency, rather than ε(λ).
    freqs = 1000 / wl  # units of 1/μm
    freqs_reduced = freqs[idx_start:idx_end]
    wl_reduced = wl[idx_start:idx_end]
    eps_reduced = eps[idx_start:idx_end]

    # Fitting parameter: number of Lorentzian terms to use in the fit
    num_lorentzians = 2

    # Number of times to repeat local optimization with random initial values.
    num_repeat = 30

    ps = np.zeros((num_repeat, 3 * num_lorentzians))
    mins = np.zeros(num_repeat)
    for m in range(num_repeat):
        # Initial values for the Lorentzian polarizability terms. Each term
        # consists of three parameters (σ, ω, γ) and is chosen randomly.
        # Note: for the case of no absorption, γ should be set to zero.
        p_rand = [10 ** (np.random.random()) for _ in range(3 * num_lorentzians)]
        ps[m, :], mins[m] = lorentzfit(p_rand, freqs_reduced, eps_reduced, 1e-25, 50000)
        ps_str = "( " + ", ".join(f"{prm:.4f}" for prm in ps[m, :]) + " )"
        print(f"iteration:, {m:3d}, ps_str, {mins[m]:.6f}")

    # Find the best performing set of parameters.
    idx_opt = np.argmin(mins)
    popt_str = "( " + ", ".join(f"{prm:.4f}" for prm in ps[idx_opt]) + " )"
    print(f"optimal:, {popt_str}, {mins[idx_opt]:.6f}")

    # Define a `Medium` class object using the optimal fitting parameters.
    E_susceptibilities = []

    for n in range(num_lorentzians):
        mymaterial_freq, mymaterial_gamma = ps[idx_opt][3 * n + 1:3 * n + 3]

        if mymaterial_freq == 0:
            mymaterial_sigma = ps[idx_opt][3 * n + 0]
            E_susceptibilities.append(
                mp.DrudeSusceptibility(
                    frequency=1.0, gamma=mymaterial_gamma, sigma=mymaterial_sigma
                )
            )
        else:
            mymaterial_sigma = ps[idx_opt][3 * n + 0] / mymaterial_freq**2
            E_susceptibilities.append(
                mp.LorentzianSusceptibility(
                    frequency=mymaterial_freq,
                    gamma=mymaterial_gamma,
                    sigma=mymaterial_sigma,
                )
            )

    Au = mp.Medium(epsilon=eps_inf, E_susceptibilities=E_susceptibilities)
    

In [ ]:
#Coupling field enhancement

import meep as mp
import numpy as np
import matplotlib.pyplot as plt



def get_incident_value(cell_size, resolution, sources, pml_layers, d):

    print("$$$$$$$$$$$$$$$$ Calculating Incident Field Strength $$$$$$$$$$$$$$$$$$$$")
    
    default_material = mp.Medium(index=1.3325)

    resolution = 4/r

    sim = mp.Simulation(
            resolution=resolution,
            cell_size=cell_size,
            boundary_layers=pml_layers,
            sources=sources,
            k_point=mp.Vector3()
        )
    
    sim.run(until=10) #10 fs

    d=0.3*d

    Ez_data_1D = np.absolute(sim.get_array(center=mp.Vector3(), size=mp.Vector3(y=d), component=mp.Ey))

    return Ez_data_1D[0]



def get_sphere_coupling_enhancement(r, gap, wvl):
    
    #default medium is water
    default_material = mp.Medium(index=1.3325)
    
    frq = 1 / wvl

    #50/r?
    resolution = 100/r
    
    dpml = 2*r
    dair = 2*r
    #defining gap between spheres. Can adjust later.
    
    pml_layers = [mp.PML(thickness=dpml)]
    
    #Fine to keep these since the system is still symmetric relative to these planes.
    #symmetries = [mp.Mirror(mp.Y)]
    
    width = 2 * (dpml + dair + r)
    height = 2 * (dpml + dair + 2*r + 0.5*gap)
    cell_size = mp.Vector3(width, height)
    
    # is_integrated=True necessary for any planewave source extending into PML
    sources = [
        mp.Source(
            mp.ContinuousSource(frequency=frq, is_integrated=True),
            center=mp.Vector3(-0.5*width + dpml, 0),
            size=mp.Vector3(0, height),
            component=mp.Hz,
        )
    ]
    
    
    
    geometry = [
        mp.Sphere(material=Au,
                  center=mp.Vector3(0, 0.5*gap + r),
                  radius=r),
        mp.Sphere(material=Au,
                  center=mp.Vector3(0, -0.5*gap - r),
                  radius=r),
    ]
    
    
    sim = mp.Simulation(
        resolution=resolution,
        cell_size=cell_size,
        boundary_layers=pml_layers,
        sources=sources,
        k_point=mp.Vector3(),
        geometry=geometry,
    )
    
    
    sim.run(until=10) #10 fs
    
    #grab data once run is finished i.e. what the system looks like at 200fs:
    d = 0.95*(height-2*dpml)
    w = width-2*dpml

    Ez_data_1D = np.absolute(sim.get_array(center=mp.Vector3(), size=mp.Vector3(y=d), component=mp.Ey))

    incident_field_value = get_incident_value(cell_size, resolution, sources, pml_layers, d)

    eps_data_x = sim.get_array(center=mp.Vector3(), size=mp.Vector3(w, d), component=mp.Dielectric)
    Ex_data = np.absolute(sim.get_array(center=mp.Vector3(), size=mp.Vector3(w, d), component=mp.Ey))

    return Ez_data_1D, incident_field_value, Ex_data, eps_data_x




In [ ]:
# Varying Gap Size

wvl = 545E-3
r = 50E-3

E_data = []  #Going to append all raw sim data arrays, to process and plot after the runs
gap_labels = []
radii = []
inc_field_vals = []
E_data_vis = []
eps_data_vis = []


gap_min =2E-3
gap_max =120E-3
gaps_far = np.linspace(gap_max, 25E-3, 15)
gaps_close = np.linspace(25E-3, gap_min, 10)
gaps = np.concatenate((gaps_far, gaps_close[1:]))
print(gaps)

index = 1

for gap in gaps:
    print(f"##################### Run {index} Starting #######################")
    E_strength, inc_field_val, Ex_data, eps_data_x = get_sphere_coupling_enhancement(r, gap, wvl)
    
    E_data.append(E_strength)
    gap_labels = np.append(gap_labels, round(gap, 2))
    radii = np.append(radii, r)
    inc_field_vals.append(inc_field_val)
    E_data_vis.append(Ex_data)
    eps_data_vis.append(eps_data_x)

    index += 1


In [ ]:
print(inc_field_vals)

FEF_values = []

index = 0

plt.figure()
for run in E_data:
    
    E_strength = run
    
    pixels = len(E_data[index])
    range = (radii[index]*5+gaps[index])/2
    space_1D = 1E3*(np.linspace(-range, +range, pixels))
    #Also converting axis to position from pixels. 100 being the resolution value.

    FEF_values.append(np.interp(0.0, space_1D, E_strength) / inc_field_vals[index])

    #plt.figure()
    plt.plot(space_1D, E_strength, "-", label=f"{str(1E3*round(float(gaps[index]),4))[:5]}")
    plt.grid(True, which="both", ls="-")
    plt.xlabel("Position, nm")
    plt.ylabel("Electric Field Strength, Vm$^{-1}$")
    plt.title(f"Two Au Nanospheres, Varying Gap Size: z Axis Field Strength (incident λ = {wvl*1E3}nm, Radius = {r*1E3}nm)")
    plt.legend(loc="upper right", title = "Gap Size, nm", fontsize = 7, title_fontsize= 8, ncol=2)
    plt.tight_layout()
    
    index+=1

plt.savefig(fname="Two Au Nanospheres, Varying Gap Size: z Axis Field Strength",
        dpi=150, 
        bbox_inches="tight")

plt.figure()
plt.plot(1E3*gaps, FEF_values, "x-g")
plt.grid(True, which="both", ls="-")
plt.xlabel("Gap size, nm")
plt.ylabel("Field Enhancement Factor, N/A")
plt.title(f"Field Enhancement Factor Against Gap Size (incident λ = {wvl*1E3}nm)")
plt.tight_layout()

plt.savefig(fname="Field Enhancement Factor Against Gap Size",
        dpi=150, 
        bbox_inches="tight")

print(1E3*gaps, FEF_values)
#Now just visualise the sphere with greatest enhancement.

index_largest_enhancement = np.argmax(FEF_values)

plt.figure()
plt.imshow(eps_data_vis[index_largest_enhancement].transpose(), interpolation='spline36', cmap='binary')
plt.imshow(E_data_vis[index_largest_enhancement].transpose(), interpolation='spline36', cmap='magma', alpha=0.9)#, vmin=vmin)
colorbar = plt.colorbar()
colorbar.set_label("Electric Field Strength, Vm$^{-1}$")

plt.xlabel("x, px")
plt.ylabel("y, px")
plt.title(f"x plane: Two Au Nanospheres (λ = {1E3*wvl}nm)")
plt.savefig(fname="x plane: Two Au Nanospheres, greatest FEF, gap",
        dpi=150, 
        bbox_inches="tight")





for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]:
    plt.figure()
    plt.imshow(eps_data_vis[i].transpose(), interpolation='spline36', cmap='binary')
    plt.imshow(E_data_vis[i].transpose(), interpolation='spline36', cmap='magma', alpha=0.9)#, vmin=vmin)
    colorbar = plt.colorbar()
    colorbar.set_label("Electric Field Strength, Vm$^{-1}$")
    
    plt.xlabel("x, px")
    plt.ylabel("y, px")
    plt.title(f"run={radii[i]}x plane: Au Nanosphere (λ = {1E3*wvl}nm)")
    plt.savefig(fname=f"r={int(i)}, x plane: Two Au Nanospheres, gap",
        dpi=150, 
        bbox_inches="tight")
    

In [ ]:
# Varying Radius

wvl = 545E-3
gap = 50E-3

E_data = []  #Going to append all raw sim data arrays, to process and plot after the runs
inc_field_vals = []
E_data_vis = []
eps_data_vis = []


r_min =15E-3
r_max =180E-3

r_big = np.linspace(r_max, 105E-3, 6)
r_peak = np.linspace(105E-3, 90E-3, 10)
r_small = np.linspace(90E-3, r_min, 9)
radii = np.concatenate((r_big, r_peak[1:], r_small[1:]))

index = 1

for r in radii:
    print(f"##################### Run {index} Starting #######################")
    E_strength, inc_field_val, Ex_data, eps_data_x = get_sphere_coupling_enhancement(r, gap, wvl)
    
    E_data.append(E_strength)
    inc_field_vals.append(inc_field_val)
    E_data_vis.append(Ex_data)
    eps_data_vis.append(eps_data_x)

    index += 1


In [ ]:
FEF_values = []
print(inc_field_vals)

index = 0

plt.figure()
for run in E_data:
    
    E_strength = run
    
    pixels = len(E_data[index])
    range = (radii[index]*5+gap)/2
    space_1D = 1E3*(np.linspace(-range, +range, pixels))
    #Also converting axis to position from pixels. 100 being the resolution value.

    FEF_values.append(np.interp(0.0, space_1D, E_strength) / inc_field_vals[index])

    #plt.figure()
    plt.plot(space_1D, E_strength, "-", label=f"{str(1E3*round(float(radii[index]),2))[:5]}")
    plt.grid(True, which="both", ls="-")
    plt.xlabel("Position, nm")
    plt.ylabel("Electric Field Strength, Vm$^{-1}$")
    plt.title(f"Two Au Nanospheres, Varying Radius: z Axis Field Strength (incident λ = {wvl*1E3}nm, Gap Size = {gap*1E3}nm)")
    plt.legend(loc="upper right", title = "Radius, nm", fontsize = 7, title_fontsize= 8, ncol=2)
    plt.tight_layout()
    #plt.show()
    
    index+=1

plt.savefig(fname="Two Au Nanospheres, Varying Radius: z Axis Field Strength",
        dpi=150, 
        bbox_inches="tight")

plt.figure()
plt.plot(1E3*radii, FEF_values, "x-g")
plt.grid(True, which="both", ls="-")
plt.xlabel("Radius, nm")
plt.ylabel("Field Enhancement Factor, N/A")
plt.title(f"Field Enhancement Factor Against Radius Size (incident λ = {wvl*1E3}nm)")
plt.tight_layout()

plt.savefig(fname="Field Enhancement Factor Against Radius Size",
        dpi=150, 
        bbox_inches="tight")

print(1E3*radii, FEF_values)

#Now just visualise the sphere with greatest enhancement.

index_largest_enhancement = np.argmax(FEF_values)

plt.figure()
plt.imshow(eps_data_vis[index_largest_enhancement].transpose(), interpolation='spline36', cmap='binary')
plt.imshow(E_data_vis[index_largest_enhancement].transpose(), interpolation='spline36', cmap='magma', alpha=0.9)#, vmin=vmin)
colorbar = plt.colorbar()
colorbar.set_label("Electric Field Strength, Vm$^{-1}$")

plt.xlabel("y, px")
plt.ylabel("z, px")
plt.title(f"x plane: Au Nanosphere (λ = {1E3*wvl}nm)")
plt.savefig(fname=f"run=x plane: Two Au Nanospheres, greatest FEF, rad",
    dpi=150, 
    bbox_inches="tight")


for index in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]:
    plt.figure()
    plt.imshow(eps_data_vis[index].transpose(), interpolation='spline36', cmap='binary')
    plt.imshow(E_data_vis[index].transpose(), interpolation='spline36', cmap='magma', alpha=0.9)#, vmin=vmin)
    colorbar = plt.colorbar()
    colorbar.set_label("Electric Field Strength, Vm$^{-1}$")
    
    plt.xlabel("y, px")
    plt.ylabel("z, px")
    plt.title(f"r={radii[index]}x plane: Au Nanosphere (λ = {1E3*wvl}nm)")
    plt.savefig(fname=f"run={index}x plane: Two Au Nanospheres, rad",
        dpi=150, 
        bbox_inches="tight")


In [ ]:
#manual for gap vary

import numpy as np
import matplotlib.pyplot as plt

wvl = 545E-3
r = 50E-3


FEF_values_man = [1.531175328271662, 1.5776426104329844, 1.6288288379475897, 1.6849860807852373, 1.748794012395692, 1.8211466819518798, 1.9018609388034964, 1.9922222812243184, 2.0991352019001255, 2.2263692910119803, 2.377310933883673, 2.561947529571016, 2.775582071620207, 3.0738478968673824, 3.4882587543060692, 3.6847819013431913, 3.900860937360002, 4.20704906726499, 4.456877405416683, 4.966562554269961, 5.2715251438871835, 6.077814085540926, 6.040190525038229, 10.5]

gaps_man = [
    120.0, 113.21428571, 106.42857143, 99.64285714, 92.85714286,
    86.07142857, 79.28571429, 72.5, 65.71428571, 58.92857143,
    52.14285714, 45.35714286, 38.57142857, 31.78571429, 25.0,
    22.44444444, 19.88888889, 17.33333333, 14.77777778, 12.22222222,
    9.66666667, 7.11111111, 4.55555556, 2.0
]



plt.figure()
plt.plot(gaps_man, FEF_values_man, "x-g")
plt.grid(True, which="both", ls="-")
plt.xlabel("Radius, nm")
plt.ylabel("Field Enhancement Factor, N/A")
plt.title(f"Field Enhancement Factor Against Gap Size (incident λ = {wvl*1E3}nm)")
plt.tight_layout()

plt.savefig(fname="MANUAL Field Enhancement Factor Against Gap Size",
        dpi=150, 
        bbox_inches="tight")
